In [1]:
!pip install tensorflow-model-optimization
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 4.1 MB/s eta 0:00:00


In [2]:
# Load the TF Flowers dataset
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
num_classes = info.features['label'].num_classes

def preprocess(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0  # Normalize
    return image, label

# Prepare training and validation datasets
train_data = dataset['train'].map(preprocess).batch(32).shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_data = dataset['train'].map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [3]:
# Define a simple CNN model
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the baseline model
model.fit(train_data, validation_data=val_data, epochs=5)

9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/5
115/115 [==============================] - 41s 96ms/step - loss: 0.7388 - accuracy: 0.7335 - val_loss: 0.4059 - val_accuracy: 0.8695
Epoch 2/5
115/115 [==============================] - 11s 81ms/step - loss: 0.3798 - accuracy: 0.8798 - val_loss: 0.3070 - val_accuracy: 0.9008
Epoch 3/5
115/115 [==============================] - 12s 83ms/step - loss: 0.3090 - accuracy: 0.8948 - val_loss: 0.2618 - val_accuracy: 0.9223
Epoch 4/5
115/115 [==============================] - 12s 82ms/step - loss: 0.2646 - accuracy: 0.9163 - val_loss: 0.2205 - val_accuracy: 0.9371
Epoch 5/5
115/115 [==============================] - 11s 73ms/step - loss: 0.2286 - accuracy: 0.9311 - val_loss: 0.1958 - val_accuracy: 0.9482


In [6]:
# Apply Structured-based pruning
def apply_structured_pruning(model):
    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

    # Define structured pruning parameters
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.2,
                                                                  final_sparsity=0.5,
                                                                  begin_step=0,
                                                                  end_step=1000)
    }

    pruned_model = prune_low_magnitude(model, **pruning_params)
    return pruned_model

# Apply pruning and recompile
pruned_model = apply_structured_pruning(model)
pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Add the UpdatePruningStep callback
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]

# Fine-tune pruned model with the callback
pruned_model.fit(train_data, validation_data=val_data, epochs=5, callbacks=callbacks) # Added callbacks here

Epoch 1/5
115/115 [==============================] - 64s 191ms/step - loss: 0.2789 - accuracy: 0.9101 - val_loss: 0.5786 - val_accuracy: 0.7948
Epoch 2/5
115/115 [==============================] - 16s 127ms/step - loss: 0.4726 - accuracy: 0.8221 - val_loss: 0.5056 - val_accuracy: 0.8202
Epoch 3/5
115/115 [==============================] - 15s 117ms/step - loss: 0.5679 - accuracy: 0.7959 - val_loss: 0.5530 - val_accuracy: 0.8082
Epoch 4/5
115/115 [==============================] - 16s 127ms/step - loss: 0.6736 - accuracy: 0.7643 - val_loss: 0.6397 - val_accuracy: 0.7711
Epoch 5/5
115/115 [==============================] - 15s 117ms/step - loss: 0.7587 - accuracy: 0.7150 - val_loss: 0.7017 - val_accuracy: 0.7368


In [7]:
# Evaluate pruned model
_, pruned_acc = pruned_model.evaluate(val_data)
print(f"Pruned Model Accuracy: {pruned_acc:.4f}")

115/115 [==============================] - 5s 40ms/step - loss: 0.7017 - accuracy: 0.7368
Pruned Model Accuracy: 0.7368
